# Transformer and Language Models

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

import torchtext, datasets, math
from tqdm import tqdm

from queue import PriorityQueue
import operator

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
# torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

torch.cuda.get_device_name(0)

cuda:0


'NVIDIA RTX A6000'

## 1. Load data - Wiki Text

We will be using wikitext which contains a large corpus of text, perfect for language modeling task.  This time, we will use the `datasets` library from HuggingFace to load.

In [3]:
import os
os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

#there are raw and preprocessed version; we used the raw one and preprocessed ourselves for fun
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')
print(dataset)

Found cached dataset wikitext (/home/chaklams/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [4]:
print(dataset['train'][333]['text'])

'''
If you try to change the index you might notice that sometimes there is no paragraph 
and rather an empty string so we will have to care of that later.
'''

 During the same time frame as the Hitchcock rumors , goaltender Curtis Sanford returned from his groin injury on November 13 . He made his first start of the season against the Boston Bruins , losing 2 – 1 in a shootout . Sanford continued his strong play , posting a 3 – 1 – 2 record , 1 @.@ 38 goals against average and .947 save percentage over his next six games . Sanford started 12 consecutive games before Steve Mason made his next start . The number of starts might not have been as numerous , but prior to the November 23 game , Mason was hit in the head by a shot from Rick Nash during pre @-@ game warm @-@ ups and suffered a concussion . Mason returned from his concussion after two games , making a start against the Vancouver Canucks . Mason allowed only one goal in the game despite suffering from cramping in the third period , temporarily being replaced by Sanford for just over three minutes . Columbus won the game 2 – 1 in a shootout , breaking a nine @-@ game losing streak to t

'\nIf you try to change the index you might notice that sometimes there is no paragraph \nand rather an empty string so we will have to care of that later.\n'

## 2. Preprocessing

### Tokenizing

Simply tokenize the given text to tokens.

In [5]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

#function to tokenize
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['text'])}  

#map the function to each example
tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})
print(tokenized_dataset['train'][333]['tokens'])

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-2f3ef63bf035b562.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-8af47333e652a54e.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6a28ab3703035b28.arrow


['during', 'the', 'same', 'time', 'frame', 'as', 'the', 'hitchcock', 'rumors', ',', 'goaltender', 'curtis', 'sanford', 'returned', 'from', 'his', 'groin', 'injury', 'on', 'november', '13', '.', 'he', 'made', 'his', 'first', 'start', 'of', 'the', 'season', 'against', 'the', 'boston', 'bruins', ',', 'losing', '2', '–', '1', 'in', 'a', 'shootout', '.', 'sanford', 'continued', 'his', 'strong', 'play', ',', 'posting', 'a', '3', '–', '1', '–', '2', 'record', ',', '1', '@', '.', '@', '38', 'goals', 'against', 'average', 'and', '.', '947', 'save', 'percentage', 'over', 'his', 'next', 'six', 'games', '.', 'sanford', 'started', '12', 'consecutive', 'games', 'before', 'steve', 'mason', 'made', 'his', 'next', 'start', '.', 'the', 'number', 'of', 'starts', 'might', 'not', 'have', 'been', 'as', 'numerous', ',', 'but', 'prior', 'to', 'the', 'november', '23', 'game', ',', 'mason', 'was', 'hit', 'in', 'the', 'head', 'by', 'a', 'shot', 'from', 'rick', 'nash', 'during', 'pre', '@-@', 'game', 'warm', '@-@

### Numericalizing

We will tell torchtext to add any word that has occurred at least three times in the dataset to the vocabulary because otherwise it would be too big.

In [6]:
## numericalizing

# Define special symbols and indices
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'], min_freq=3, specials=special_symbols)   

vocab.set_default_index(vocab['<unk>'])   
print(len(vocab))                         
print(vocab.get_itos()[:10])       

29475
['<unk>', '<pad>', '<sos>', '<eos>', 'the', ',', '.', 'of', 'and', 'in']


## 3. Prepare the batch loader

### Prepare data

Given "Chaky loves eating at AIT", and "I really love deep learning", and given batch size = 3, we will get three batches of data "Chaky loves eating at", "AIT `<eos>` I really", "love deep learning `<eos>`".  

In [7]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:
        if example['tokens']:         
            #appends eos so we know it ends....so model learn how to end...                             
            tokens = example['tokens'].append('<eos>')   
            #numericalize          
            tokens = [vocab[token] for token in example['tokens']] 
            data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size #get the int number of batches...
    data = data[:num_batches * batch_size] #make the batch evenly, and cut out any remaining                      
    data = data.view(batch_size, num_batches)          
    return data #[batch size, bunch of tokens]

In [8]:
batch_size = 128
train_data = get_data(tokenized_dataset['train'], vocab, batch_size)
valid_data = get_data(tokenized_dataset['validation'], vocab, batch_size)
test_data  = get_data(tokenized_dataset['test'], vocab, batch_size)

## 4. Modeling 

In [9]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [10]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        #x = [batch size, seq len, hid dim]
        
        return x

Here I am using Batched Beam Search, where instead of feeding each hypothesis one by one, which takes a lot of time;  I simply concat everything into one list and feed them all at once, which is much faster.

In [11]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, 
                 pf_dim, dropout, device, pad_idx, max_length = 100):
                
        super().__init__()
        
        self.device = device
        self.output_dim = output_dim
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        self.pad_idx = pad_idx
    
    def make_mask(self, x):
        
        #x = [batch size, len]
        
        pad_mask = (x != self.pad_idx).unsqueeze(1).unsqueeze(2)
        #pad_mask = [batch size, 1, 1, len]
        
        x_len = x.shape[1]
        
        sub_mask = torch.tril(torch.ones((x_len, x_len), device = self.device)).bool()
        #sub_mask = [len, len]
            
        mask = pad_mask & sub_mask
        #mask = [batch size, 1, len, len]
        
        return mask 
    
    def forward(self, x):
        
        #x = [batch size, len]
                
        batch_size = x.shape[0]
        x_len      = x.shape[1]
        
        #get mask here since we remove seq2seq class
        mask   = self.make_mask(x)
        #mask = [batch size, 1, len, len]

        pos = torch.arange(0, x_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)          
            
        x = self.dropout((self.tok_embedding(x) * self.scale) + self.pos_embedding(pos))
        #x = [batch size, len, hid dim]
        
        for layer in self.layers:
            x, attention = layer(x, mask)
        
        #x = [batch size, len, hid dim]
        #attention = [batch size, n heads, len, len]
        
        output = self.fc_out(x)
        #output = [batch size, len, output dim]
            
        return output, attention

    def beam_decode(self, penalty_alpha = 0.9, max_length = 5, beam_size = 5):
        
        # Start with SOS Harry Potter is
        prompt = 'Harry Potter is '
        
        tokens = tokenizer(prompt)
        indices = [SOS_IDX] + [vocab[t] for t in tokens]

        decoder_input = torch.Tensor([indices]).long().to(device)
        #decoder_input: [batch size, len] = [1, 1]
        scores = torch.Tensor([0.]).to(device)
        #scores: [1]
        
        for i in range(max_length):
            
            # print(f"========Length: {i}")
            
            # Decoder prediction
            logits, _ = self.forward(decoder_input)
            #[beam_size, current dec len=i, vocab_size]
                        
            logits = logits[:, -1] 
            # Last sequence step: [beam_size, current dec len=i, vocab_size] => [beam_size, vocab_size]
            
            # print(f"{logits.shape=}")

            # Softmax
            # Log softmax is better, since beam search accumulates probability
            # if simply softmax, the probability can get too small and then become unstable
            log_probs = torch.log_softmax(logits, dim=1)
    
            # Add length penalty, otherwise, always very short sentence will win...
            penalty   = ((5 + (i+1)) / (5 + 1)) ** penalty_alpha #see https://arxiv.org/abs/1609.08144
            log_probs = log_probs / penalty
            
            # print(f"{decoder_input[:, -1]=}")
            
            # Update score where EOS has not been reached
            log_probs[decoder_input[:, -1]==EOS_IDX, :] = -2 #discouraged it to end
            log_probs[decoder_input[:, -1]==UNK_IDX, :] = -10 #very discouraged to spit out unk
            scores = scores.unsqueeze(1) + log_probs 
            # scores: [beam_size, vocab_size]
            # log_probs: [beam_size, vocab_size]

            # print(f"{log_probs.shape=}")
            # print(f"{scores.shape=}")
            #log_probs: torch.Size([1, 29475])
            #scores.shape=torch.Size([1, 29475])
            
            # Flatten scores from [beams, vocab_size] to [beams * vocab_size] to get top k, and reconstruct beam indices and token indices
            # Since we flatten it, we have to retrieve the actual beam indices and token_indices using floor division and remainder
            # You can try on paper; it will make sense
            scores, indices = torch.topk(scores.reshape(-1), beam_size) #scores: [beam_size]; #indices: [beam_size]
            beam_indices  = torch.divide   (indices, self.output_dim, rounding_mode='floor') # indices // vocab_size
            token_indices = torch.remainder(indices, self.output_dim)                        # indices %  vocab_size
            
            # print(f"{scores=}")
            # print(f"{indices.shape=}")
            
            # print(f"{indices=}")
            # print(f"{beam_indices=}")
            # print(f"{token_indices=}")
            
            # Build the next decoder input
            # For efficiency, the trick is to concatenate all hypotheses into one string and sent to decoder at once
            # We can later chop it ...
            next_decoder_input = []
            for beam_index, token_index in zip(beam_indices, token_indices):
                # print(f"{beam_index=}")
                prev_decoder_input = decoder_input[beam_index]
                # print(f"{prev_decoder_input=}")
                if prev_decoder_input[-1]==EOS_IDX:
                    token_index = EOS_IDX # once EOS, always EOS
                token_index = torch.LongTensor([token_index]).long().to(device)
                next_decoder_input.append(torch.cat([prev_decoder_input, token_index]))
                # print("here: " + " ".join([vocab.lookup_token(i) for i in next_decoder_input[-1]]) + "; score: " + str(scores[beam_index].item()))
            decoder_input = torch.vstack(next_decoder_input)
            
            # print(f"{decoder_input=}")
            
             # If all beams are finished, and the length is at least 5, exit
            if i > 5:
                if (decoder_input[:, -1]==EOS_IDX).sum() == beam_size:
                    break
                
        # convert the top scored sequence to a list of text tokens
        decoder_output, _ = max(zip(decoder_input, scores), key=lambda x: x[1])
        decoder_output = decoder_output[1:].cpu().numpy() # remove SOS
        
        return [vocab.lookup_token(i) for i in decoder_output if i != EOS_IDX] # remove EOS if exists

In [12]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)        
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        
        #x = [batch size, len, hid dim]
        #mask = [batch size, 1, len, len]
        
        #multi attention, skip and then norm
        _x, attention = self.self_attention(x, x, x, mask)
        x = self.self_attn_layer_norm(x + self.dropout(_x))
        #x = [batch size, len, hid dim]
        #attention = [batch size, n heads, len, len]
    
        #positionwise feedforward
        _x = self.positionwise_feedforward(x)
        x = self.ff_layer_norm(x + self.dropout(_x))
        #x = [batch size, len, hid dim]
        
        return x, attention

## 5. Training 

In [13]:
vocab_size = len(vocab)
hid_dim    = 256                
dec_layers = 3               
dec_heads  = 8
dec_pf_dim = 512
dec_dropout = 0.1     
lr = 1e-3                     

In [14]:
model = Decoder(vocab_size, hid_dim, dec_layers, dec_heads, dec_pf_dim, dec_dropout, device, PAD_IDX).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 16,727,587 trainable parameters


In [15]:
def get_batch(data, seq_len, idx):
    #data #[batch size, bunch of tokens]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [16]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    # data #[batch size, bunch of tokens]
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]  #we need to -1 because we start at 0
    num_batches = data.shape[-1]
        
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        
        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, _ = model(src)               

        #need to reshape because criterion expects pred to be 2d and target to be 1d
        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [17]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]
    
    decoded_batch_list = []

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            #target = [batch size, dec len]

            batch_size= src.shape[0]
            prediction, _ = model(src)
            #prediction = [batch size, dec len, output_dim]
            
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
            
    #decoding using beam_search as example (you don't need to put here, because beam_search is for intference)
    decoded_batch = model.beam_decode()
    print("Sample beam sentence: " + " ".join(decoded_batch))
            
    return epoch_loss / num_batches

Here we will be using a `ReduceLROnPlateau` learning scheduler which decreases the learning rate by a factor, if the loss don't improve by a certain epoch.

In [18]:
n_epochs = 5
seq_len  = 50 #<----decoding length
clip    = 0.25

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(n_epochs):
    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)

    lr_scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-tr_lm.pt')

    print(f'Epoch: {epoch+1}:')
    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Sample beam sentence: harry potter is a few years .
Epoch: 1:
	Train Perplexity: 530.221
	Valid Perplexity: 295.877


Sample beam sentence: harry potter is used in the most recent
Epoch: 2:
	Train Perplexity: 265.409
	Valid Perplexity: 241.811


Sample beam sentence: harry potter is one of the most common
Epoch: 3:
	Train Perplexity: 193.361
	Valid Perplexity: 220.590


Sample beam sentence: harry potter is one of the most common
Epoch: 4:
	Train Perplexity: 154.279
	Valid Perplexity: 213.680


Sample beam sentence: harry potter is the most common starling of
Epoch: 5:
	Train Perplexity: 130.375
	Valid Perplexity: 211.744


## 6. Testing

In [19]:
model.load_state_dict(torch.load('best-val-tr_lm.pt',  map_location=device))
test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
print(f'Test Perplexity: {math.exp(test_loss):.3f}')

Sample beam sentence: harry potter is the most common starling of
Test Perplexity: 194.983


## 7. Real-world inference

Here I only use pure sampling.  You may want to put the beam search here and compare.  I will leave them as your practice.

In [20]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, _ = model(src)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input
            
            #####################################################################
            #I only do pure sampling....
            #you may want to compare here with top-k, top-p, and beam search here
            #####################################################################

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [21]:
prompt = 'Harry Potter is '
max_seq_len = 30
seed = 0

#smaller the temperature, more diverse tokens but comes 
#with a tradeoff of less-make-sense sentence
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
harry potter is the most popular culture .

0.7
harry potter is the man of the kings and the year of a few @-@ old vic @-@ old vic scene , and the third @-@ year @-@ old testament , and was

0.75
harry potter is the man of the kings and the year of a few @-@ old vic @-@ old vic scene , and the third @-@ year @-@ old testament , as follows

0.8
harry potter is the man of the kings and the year in a 2 – 0 – 11 season record title , and the third season . in 2012 , the crimson tide

1.0
harry potter is the spatial man of the kings and the year in a cruciform province .

